## Tarea 10.2

Escriba un código que calcule integrales triples usando una cuadratura gaussiana, y úselo para calcular en coordenadas cartesianas la carga total al interior de una esfera con densidad $\rho = r$ con $r \leq 1$. Compare su resultado con el valor exacto.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

En el caso 2d el integral puede escribirse como:
$$
\int_a^b dx \int_{c(x)}^{d(x)} dy\; f(x,y)
$$

Con esta definición podemos generalizar para 3d:
$$
\int_a^b dx \int_{c(x)}^{d(x)} dy \int_{e(x,y)}^{f(x,y)} dz\;f(x,y,z)
 $$

Con $c(x)$,$d(x)$,$e(x,y)$ y $f(x,y)$ funciones que cambian los limites de integración tal que el algoritmo de cuadratura gaussiana puede ser aplicado.

La cuadradura Gaussiana esta definida en [-1,1], por lo cual es necesario cambiar nuestros limites y variables de integración tal que sea valido ocupar el metodo de cuadratura Gaussiana.

El primer cambio es que x este definida por $t(x)=\frac{b-a}{2}x + \frac{b+a}{2}$.

Ahora bien debemos cambiar los limites de integración de $y$ de tal forma que quede entre $-1$ y $1$.

Como se puede pensar en el integral como un operador podemos hacer una analogia similar con la cuadratura Gaussiana, por ende el cambio para el integral de $y$ sera el mismo que el de $x$, lo mismo sucede con $z$. Entonces:

$$
  \int_a^b dx \int_{c(x)}^{d(x)} dy \int_{e(x,y)}^{f(x,y)} dz\;f(x,y,z)  \approx \sum_{i=1}^{n} \sum_{j=1}^{m} \sum_{k=1}^{l} c_i c_j c_k f(x_i,x_j,x_k)  
$$


In [2]:
def gauss_quad_7_3d(f, a=-1, b=1, c=lambda x: -1, d=lambda x:1, e=lambda x,y: -1, fg=lambda x,y: 1):
    """
    Aproxima el integral de la función de tres variables f con un metodo de cuadradatura Gaussiana de grado 7.
    
    Parametros:
    ----------
    f: function
        Función a integrar.
    a: float, opcional
        Limite inferior para x. Predefinido como -1.
    b: float, opcional
        Limite superior para x. Predefinido como 1.
    c: function, opcional
        Función de x que define como cambia el limite inferior para y. De manera
        predeterminada es la función f(x)=-1. 
    d: function, opcional
        Función de x que define como cambia el limite superior para y. De manera
        predeterminada es la función f(x)=1.
    e: function, opcional
        Función de x y de c(y) que define como cambia el limite inferior de z.
        Tiene un valor predeterminado de una función de la forma f(x)=-1.
    fg: function, opcional
        Función de x y de d(y) que define como cambia el limite supeior de z.
        Tiene un valor predeterminado de una función de la forma f(x)=1.        
    """
    
    #Coeficientes 
    
    cg = [0.1012285362903706, 0.2223810344533744, 0.3137066458778874, 0.3626837833783621, 0.3626837833783621,0.3137066458778874, 0.2223810344533744, 0.1012285362903706]

    rg = [-0.9602898564975363, -0.7966664774136267, -0.5255324099163290, -0.1834346424956498,0.1834346424956498, 0.5255324099163290, 0.7966664774136267, 0.9602898564975363]
    result = 0 #acumulador
    
    #Cambio limites
    
    def t(x,a,b):
        h1 = (b-a)/2
        h2 = (a+b)/2
        return h1*x + h2
    
    #Constante para el cambio de limites
    
    def kg(a,b):
        return (b - a)/2
    
    kx = kg(a,b);
    l = len(cg)
    for i in range(l):
        
        x = t(rg[i],a,b);
        c1 = c(x);
        d1 = d(x);
        ky = kg(c1,d1);
    
        for j in range(l):
            
            y = t(rg[j],c1,d1);
            e1 = e(x,y);
            f1 = fg(x,y);
            
            kz = kg(e1,f1);
            for k in range(l):
                
                z = t(rg[k],e1,f1);
                result += cg[i]*cg[j]*cg[k]*kz*ky*kx*f(x, y, z);
            
    return result

Probamos nuestro codigo con un integral conocido. Por ejemplo $\int_{0}^{1} dx\int_{0}^{1}dy\int_{0}^{1}dz$ xyz. Su valor exacto es $\frac{1}{8}=0.125$.

In [3]:
def f(x,y,z):
    return x*y*z

In [4]:
#Integramos con los limites 0 y 1 para todas las variables
approx_gauss = gauss_quad_7_3d(f, 0, 1, lambda x: 0, lambda x: 1,lambda x,y: 0, lambda x,y: 1);

In [5]:
exact = 1/8
approx_gauss

0.12499999999999793

In [6]:
err_abs = abs(exact-approx_gauss)/(exact);
err_abs

1.6542323066914832e-14

Tenemos un error del orden de $10^{-14}$. Ya con nuestra función bien definida y demostrada su presición realizamos el calculo para la esfera maciza de radio 1 y densidad constante de $0.1$.

In [7]:
#Función de densidad 
def p(x,y,z):
    return 0.1 #Constante para todo x,y,z

Ahora bien debemos calcular los limites de integración en coordenadas cartesianas para una esfera de radio $R$.

Si proyectamos la esfera en el eje x es evidente que $-R\leq x \leq R$, por ende ya tenemos el limite de integración para $x$. Se cumple igualmente que $0\leq z^2 + y^2 \leq R^2 - x^2$.

Luego si proyectamos un circulo en el plano $xy$ tendremos que el radio de este varia entre $-\sqrt{R^2 - x^2}$ y $\sqrt{R^2 - x^2}$. Por ultimo si proyectamos ahora desde el plano $xy$ a $z$ tendremos que $-\sqrt{R^2 - x^2 - y^2}$ y $\sqrt{R^2 - x^2 - y^2}$. Entonces nuestro integral queda como:

$$
    \int_{-R}^{R} \; dx \int_{-\sqrt{R^2 - x^2}}^{\sqrt{R^2 - x^2}}\; dy\int_{-\sqrt{R^2 - x^2 - y^2}}^{\sqrt{R^2 - x^2 - y^2}} \; dz \; \rho(x,y,z)
$$

In [8]:
R=1 #Radio de la esfera

#Definimos nuestra función de densidad
def rho(x,y,z):
    return 0.1 #Densidad constante 

Este integral es facil de calcular analiticamente y tiene un valor de $\frac{4}{3}\pi R^3\rho \rightarrow \frac{4}{30}\pi$

In [9]:
exact = 4/30 * np.pi 
exact

0.41887902047863906

In [10]:
approx_gauss_sphe = gauss_quad_7_3d(rho,-R,R,lambda x: -np.sqrt(R**2 - x**2), lambda x: np.sqrt(R**2 - x**2),lambda x,y: -np.sqrt(R**2 - x**2 - y**2),lambda x,y: np.sqrt(R**2 - x**2 - y**2))
approx_gauss_sphe

0.4192413931002175

In [11]:
err_abs = abs(exact - approx_gauss_sphe)/exact
print(f'El error cometido con la aproximación es aprox. {round(err_abs,6)*100}%')

El error cometido con la aproximación es aprox. 0.0865%


* Los coeficientes para la cuadratura gaussiana de orden 7 fueron obtenidos de J. Keesling, Gaussian Quadrature. University of Florida, 2014 [https://people.clas.ufl.edu/kees/files/GaussianQuadrature.pdf].